This is an example pipeline to train a siamsese model on to the individuals

In [ ]:
from src import ds_generator, models, triplet_eval
import util
import tensorflow as tf

In [ ]:
batch_size = 64
train_ds = ds_generator.DS_Generator().generate_individual_data(util.util.TRAIN_SPECIES_DF,
                                                                        batch_size=batch_size,
                                                                        augment=True,
                                                                        val_split=0.1,
                                                                        seed = util.INDIVIDUMS_SEED)

In [ ]:
# load model you man to train, for example:
model = models.return_new_siamese_model()

# callbacks for logs + model checkpoints
log_dir = util.SAVING_PATH + model.name + "/logs"

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

checkpoint_path = util.SAVING_PATH + model.name+ "/saves/" +"/cp-{epoch:04d}.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True)

In [ ]:
# fit model
model.fit(train_ds,
          epochs=100,
          callbacks=[cp_callback,tensorboard_callback])

In [ ]:
# Now we can evaluate our model

# first we need all the datasets for evaluation
train_ds, val_ds, train_df, val_df =  ds_generator.DS_Generator().generate_individual_data(util.TRAIN_SPECIES_DF,
                                                                        batch_size=batch_size,
                                                                        augment=True,val_split=0.1,
                                                                        seed = util.INDIVIDUMS_SEED,
                                                                        return_eval_data=1)
# also all the data of the whales with only 1 image
new_whales_ds,new_whales_df = ds_generator.DS_Generator().generate_single_individuals_ds(util.TRAIN_SPECIES_DF,batch_size)

# then we need to specify an folder where our evaluation images will be saved to
folder_path ="some_path"
# and then a name for the model
name = "our model att epoch xy"

# run eval
triplet_eval.create_triplet_eval(model, train_ds, val_ds, train_df, val_df, new_whales_ds, new_whales_df,folder_path,name)